<table>
<tr>
<td width=15%><img src="./img/UGA.png"></img></td>
<td><center><h1>Introduction à la Recherche Opérationelle</h1><br/>M2 Statistique Science des Données (SSD)</center></td>
<td width=15%>
<a href="https://www-ljk.imag.fr/membres/Anatoli.Iouditski/" style="font-size: 16px; font-weight: bold">Anatoli Juditsky</a><br/>
<a href="http://www.iutzeler.org" style="font-size: 16px; font-weight: bold">Franck Iutzeler</a><br/>
 </td>
</tr>
</table>

<br/><br/>
<center><a style="font-size: 30pt; font-weight: bold">TP 0 - Introduction à CVXPY </a></center>
<br/><br/>

# 1- Le package CVXPY

**CVXPY** est un package Python permettant la résolution de problèmes d'optimisation une fois ceux-ci formulés correctement. 

Le site de CVXPY [https://www.cvxpy.org/](https://www.cvxpy.org/) fournit de nombreuses ressources, notamment des exemples et l'API. *C'est votre ressource principale si vous n'arrivez pas à formuler un problème ou avez un bug*.

## Installation


La manière la plus simple de l'installer est d'utiliser ``pip``
    
        pip install --user cvxpy
        
Par exemple en exécutant la ligne de commande suivante:

In [1]:
!pip install --user cvxpy

*Attention:* Sous windows, vous devrez également installer `Visual Studio build tools for Python 3` , consultez https://www.cvxpy.org/install/

In [2]:
import cvxpy as cp
import numpy as np

Si l'installation est ok, les deux lignes ci-dessus ne devraient pas retourner d'erreur.

# 2- Un premier exemple


Considérons le problème de régression linéaire
$$ Y=X\beta^\star+\epsilon$$
où $Y$ est un vecteur de taille $n=20$, $X$ est une matrice de taille $n\times p = 20\times 10$ et $\beta^\star =[-4,..,-1,0,1,..,5]$ est un vecteur de taille $p=10$; finalement, epsilon est un vecteur aléatoire Gaussien de taille $n=20$ et de variance $0.01$.

In [3]:
n = 20
p = 10
beta_star = np.array(range(-4,6))   # beta is just -4 through 5.

X = np.random.randn(n,p)
Y = np.dot(X,beta_star) + 0.1*np.random.randn(n)

L'estimateur du maximum de vraisemblance de $\hat{\beta}$ sachant $Y$ et $X$ est le vecteur minimisant en $\beta$ la *fonction objectif*
$$ L(\beta) = \| X \beta - Y \|_2^2 $$

## Resolution avec CVXPY

* **Étape 1.** Définir la variable selon laquelle minimiser    

In [4]:
beta = cp.Variable(p) # p est la taille de la variable, définie ci-dessus

`beta` est désormais une *variable cvxpy*, ce n'est pas un nombre mais un objet qui sert à écrire notre objectif.

* **Étape 2.** Définir la fonction objectif

In [5]:
L = cp.sum_squares(X @ beta - Y) # L est la fonction ci-dessus 

Ici, la variable `L` est bien une *fonction pour CVXPY* comme son expression fait intervenir `beta` qui est une *variable cvxpy*. Ces fonctions peuvent faire intervenir uniquement les opérations gérées par CVXPY, voir https://www.cvxpy.org/tutorial/functions/index.html .  

*Attention :* pour écrire $X \beta$, il faut écrire `X @ beta` (ou `X * beta` sur les version plus anciennes) afin que CVXPY comprenne qu'il s'agit d'une multiplication.

* **Étape 3.** Créer le problème à résoudre

D'abord, on précise l'objectif qui est minimiser `L` (sous entendu par rapport à la variable  `beta` ).

In [6]:
objective = cp.Minimize(L)

Ensuite, on créé le problème a proprement parler.

In [7]:
prob = cp.Problem(objective)

* **Étape 4.** Le résoudre!

In [8]:
prob.solve()

0.0559828547301886

* **Étape 5.** Analyser le résultat

Après l'appel de la fonction `solve`, la variable `prob` contient (entre autres) les attributs suivants:
* `status` : est-ce que la solution trouvée est optimale
* `value` : valeur optimale de L 
* `solver_stats.solve_time` : le temps mis à trouver une solution

et la variable `beta` contient notamment la valeur optimal:
* `value` : valeur optimale (de la variable précisée)

In [9]:
prob.status

'optimal'

In [10]:
prob.value

0.0559828547301886

In [11]:
prob.solver_stats.solve_time

0.0004947

In [12]:
print("\nThe optimal value is", prob.value)
print("The optimal beta is")
print(beta.value)
print("The norm of the residual is ", cp.norm(X@beta - Y, p=2).value)


The optimal value is 0.0559828547301886
The optimal beta is
[-3.97078039 -2.96008755 -1.97658031 -0.99637114 -0.03116156  1.01701464
  2.02619344  2.96101783  3.98596306  5.06857504]
The norm of the residual is  0.23660696255644847


## Conclusion

On a donc résolu le problème de régression linéaire assez facilement. Cependant, comme nous alons le voir maintenant, CVXPY permet aisément de modifier le problème, en ajoutant par exemple des contraintes.

# 3- Un problème avec contraintes

Supposons que nous voulons maintenant résoudre le même problème de minimiser
$$ L(\beta) = \| X \beta - Y \|_2^2 $$
mais avec la contrainte supplémentaire que les composantes de $\beta$ doivent toute être positives et que leur somme doive être égale à 10.


Suivons les même étapes que précedemment.

* **Étape 1.** Définir la variable selon laquelle minimiser    

In [13]:
beta = cp.Variable(p)  # p est la taille de la variable, définie ci-dessus

* **Étape 2.** Définir la fonction objectif *et les contraintes*

In [14]:
L = cp.sum_squares(X @ beta - Y) # L est la fonction ci-dessus 

In [15]:
contrainte1 = beta >= 0.0

In [16]:
contrainte2 = cp.sum(beta) == 1

* **Étape 3.** Créer le problème à résoudre *avec les contraintes*

In [17]:
objective = cp.Minimize(L) # L'objectif de minimisation de change pas...

In [18]:
prob = cp.Problem(objective,constraints=[contrainte1,contrainte2]) # ... mais le problème a des contraintes

* **Étape 4.** Le résoudre!

In [23]:
prob.solve()

1398.4053793303342

* **Étape 5.** Analyser le résultat

In [24]:
prob.status

'optimal'

In [25]:
betaHat2 = beta.value
betaHat2

array([-2.83074454e-15,  1.23293719e-14,  1.14587477e-14,  2.75220100e-15,
       -5.19176618e-15,  9.44196849e-15,  8.13199231e-15,  2.48699723e-16,
        7.86072486e-01,  2.13927514e-01])

Les valeurs sont bien positives (aux erreurs numériques près)

In [26]:
np.sum(betaHat2)

np.float64(1.0000000000000424)

La somme des coefficients est bien égale à 1

## Variables duales


CVXPY retourne également les variables duales associées aux contraintes. `prob.constraints[0].dual_value` contient les variables duales associées à la première contrainte de la liste, `prob.constraints[1].dual_value` celles de la deuxième, etc. 

In [27]:
lambda1 = prob.constraints[0].dual_value
lambda1

array([339.5389489 , 177.87278929, 313.62681496, 186.21229738,
       121.19508322, 151.91848319,  69.74915913,  11.63567976,
         0.        ,   0.        ])

In [29]:
print("value \t dual of constraint 1")
for i in range(p):
    print("{:2.4f}  \t {:4.2f}".format(betaHat2[i],lambda1[i]))

value 	 dual of constraint 1
-0.0000  	 339.54
0.0000  	 177.87
0.0000  	 313.63
0.0000  	 186.21
-0.0000  	 121.20
0.0000  	 151.92
0.0000  	 69.75
0.0000  	 11.64
0.7861  	 0.00
0.2139  	 0.00


In [30]:
prob.constraints[1].dual_value

array(107.42695458)